In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,ConfusionMatrixDisplay
import openpyxl


pd.options.mode.copy_on_write = True

In [5]:
df = pd.read_csv('fraudTrain.csv')

In [7]:
df.drop(columns=['ID','trans_num'],inplace=True)

# Exploring *Names* features

In [8]:
df['Name'] = df['firstName'] + ' ' + df['lastName']

In [10]:
print(df['Name'].nunique())
print(df['firstName'].nunique())
print(df['lastName'].nunique())

954
347
478


In [13]:
fraudulent_df = df.loc[df['is_fraud'] == 1]

In [15]:
fraudulent_df['Name'].nunique()

575